In [ ]:
import pymc3 as pm
import pandas as pd
import theano.tensor as tt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
import seaborn as sns
%matplotlib inline

In [ ]:
with pm.Model():
    skill = pm.Gamma('skill', alpha=1, beta=1)
    performance = pm.NegativeBinomial('performance', skill, alpha=1)

In [ ]:
df_all = pd.read_csv(r'D:\Data\BFTrader\football_training_set.csv')
df_all['home_team'] = df_all['home']
df_all['away_team'] = df_all['away']
df_all['home_score'] = df_all['homeGoals']
df_all['away_score'] = df_all['awayGoals']

In [ ]:
df = df_all[['home_team', 'away_team', 'home_score', 'away_score']]
teams = list(set(list(df.home_team) + list(df.away_team.unique())))
teams = pd.DataFrame(teams, columns=['team'])
teams['i'] = teams.index

df = pd.merge(df, teams, left_on='home_team', right_on='team', how='left')
df = df.rename(columns = {'i': 'i_home'}).drop('team', 1)
df = pd.merge(df, teams, left_on='away_team', right_on='team', how='left')
df = df.rename(columns = {'i': 'i_away'}).drop('team', 1)

observed_home_goals = df.home_score.values
observed_away_goals = df.away_score.values

home_team = df.i_home.values
away_team = df.i_away.values

num_teams = len(teams)
num_games = len(home_team)

g = df.groupby('i_away')
att_starting_points = np.log(g.away_score.mean())
g = df.groupby('i_home')
def_starting_points = -np.log(g.away_score.mean())

In [ ]:
with pm.Model() as model:
    # global model parameters
    home = pm.Flat('home')
    sd_att = pm.HalfStudentT('sd_att', nu=3, sd=2.5)
    sd_def = pm.HalfStudentT('sd_def', nu=3, sd=2.5)
    intercept = pm.Flat('intercept')

    # team-specific model parameters
    atts_star = pm.Normal("atts_star", mu=0, sd=sd_att, shape=num_teams)
    defs_star = pm.Normal("defs_star", mu=0, sd=sd_def, shape=num_teams)

    atts = pm.Deterministic('atts', atts_star - tt.mean(atts_star))
    defs = pm.Deterministic('defs', defs_star - tt.mean(defs_star))
    home_theta = tt.exp(intercept + home + atts[home_team] + defs[away_team])
    away_theta = tt.exp(intercept + atts[away_team] + defs[home_team])

    # likelihood of observed data
    home_points = pm.Poisson('home_points', mu=home_theta, observed=observed_home_goals)
    away_points = pm.Poisson('away_points', mu=away_theta, observed=observed_away_goals)
    

In [ ]:
with model:
    trace = pm.sample(1000, tune=1000, cores=3)

In [ ]:
pm.traceplot(trace, varnames=['intercept', 'home', 'sd_att', 'sd_def']);

In [ ]:
bfmi = pm.bfmi(trace)
max_gr = max(np.max(gr_stats) for gr_stats in pm.gelman_rubin(trace).values())
(pm.energyplot(trace, legend=False, figsize=(6, 4))
   .set_title("BFMI = {}\nGelman-Rubin = {}".format(bfmi, max_gr)));

In [ ]:
df_hpd = pd.DataFrame(pm.stats.hpd(trace['atts']),
                      columns=['hpd_low', 'hpd_high'],
                      index=teams.team.values)
df_median = pd.DataFrame(pm.stats.quantiles(trace['atts'])[50],
                         columns=['hpd_median'],
                         index=teams.team.values)
df_hpd = df_hpd.join(df_median)
df_hpd['relative_lower'] = df_hpd.hpd_median - df_hpd.hpd_low
df_hpd['relative_upper'] = df_hpd.hpd_high - df_hpd.hpd_median
df_hpd = df_hpd.sort_values(by='hpd_median')
df_hpd = df_hpd.reset_index()
df_hpd['x'] = df_hpd.index + .5

fig, axs = plt.subplots(figsize=(10,4))
axs.errorbar(df_hpd.x, df_hpd.hpd_median,
             yerr=(df_hpd[['relative_lower', 'relative_upper']].values).T,
             fmt='o')
axs.set_title('HPD of Attack Strength, by Team')
axs.set_xlabel('Team')
axs.set_ylabel('Posterior Attack Strength')
_= axs.set_xticks(df_hpd.index + .5)
_= axs.set_xticklabels(df_hpd['index'].values, rotation=45)

In [ ]:
with model:
    pp_trace = pm.sample_posterior_predictive(trace, samples=10000)

In [ ]:
pp_trace.keys()

In [ ]:
home_win = (pp_trace['home_points'] > pp_trace['away_points']) * 1
draw = (pp_trace['home_points'] == pp_trace['away_points']) * 1
away_win = (pp_trace['home_points'] < pp_trace['away_points']) * 1
df['home_win_pred'] = home_win.mean(axis=0)
df['draw_pred'] = draw.mean(axis=0)
df['away_win_pred'] = away_win.mean(axis=0)

In [ ]:
df[(df['home_team'] == 'Southampton') & (df['away_team'] == 'Crystal Palace')]

In [ ]:
trace['atts'].shape